# Bing News API Real Time Data Collection

In [1]:
import json
import os
import requests
import time
from datetime import datetime

In [2]:
subscriptionKey = os.environ['BING_SEARCH_V7_SUBSCRIPTION_KEY']
endpoint = "https://api.bing.microsoft.com/v7.0/news"

# Construct a request
query = ""
categories = ["World", "Business", "Sports", "Science"]
count = 100
freshness = "Day"
mkt = 'en-US'
data_file_path = os.path.abspath(os.path.join(os.pardir,'data','bing_api_json'))

for category in categories:
    params = {'q': query, 'mkt': mkt, 'category': category, 'count': count, 'freshness': freshness}
    headers = {'Ocp-Apim-Subscription-Key': subscriptionKey}
    file_name = datetime.today().strftime('%Y%m%d') + "_" + category
    
    # Call the API
    try:
        response = requests.get(endpoint, headers=headers, params=params)
        response.raise_for_status()
        #write json string to file
        with open(os.path.join(data_file_path,file_name+'.json'), 'w') as json_file:
          json.dump(response.json(), json_file)
    except Exception as ex:
        raise ex

    time.sleep(1)
    #free account offers only 3 requests per second

# Real Time Data Cleaning

In [3]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import string
from num2word import word
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
import os

In [4]:
data_file_path = os.path.abspath(os.path.join(os.pardir,'data','bing_api_json',''))
mac = '/'
# windows = '\\'

#dict of json files from bing news api
dict = {'sports':[datetime.today().strftime('%Y%m%d')+'_Sports.json'],
        'world':[datetime.today().strftime('%Y%m%d')+'_World.json'],
        'business':[datetime.today().strftime('%Y%m%d')+'_Business.json'],
        'science_and_technology':[datetime.today().strftime('%Y%m%d')+'_Science.json']}

df_list = []

#iterates through each json file and stores as a dataframe in a list
for k, v in dict.items():
    for i in v:
        init_df = pd.read_json(data_file_path+mac+i)
        df = json_normalize(init_df['value'])
        df = df[['name','description']]
        df.insert(0, 'News Category', k)
        df_list.append(df)
        df.head()

#concatenates list of dataframes into one dataframe
data = pd.concat(df_list,axis=0)
#renames column title to match other data
data = data.rename(columns={'name':'Title', 'description': 'Description'})
data.shape
data.head()

#Below is Fengling's code unedited except adding comments
#Remove Punctuation and Stopwords
data['Title'] = data['Title'].str.translate(str.maketrans('','',string.punctuation)).str.lower()
data['Description'] = data['Description'].str.translate(str.maketrans('','',string.punctuation)).str.lower()


def convert_num_to_word(words):
    result = []
    for w in words:
        if w.isnumeric():
            result.extend(map(lambda x: x.lower(),word(w).split()))
        else:
            result.append(w)
    return result

data['Title'] = data['Title'].str.split().apply(convert_num_to_word)
data['Description'] = data['Description'].str.split().apply(convert_num_to_word)


def remove_stopword(words):
    result = []
    for word in words:
        if word not in STOPWORDS:
            result.append(word)
    return result

data['Title'] = data['Title'].apply(remove_stopword)
data['Description'] = data['Description'].apply(remove_stopword)


def remove_single_character(words):
    result = []
    for word in words:
        if len(word) > 1:
            result.append(word)
    return result

data['Title'] = data['Title'].apply(remove_single_character)
data['Description'] = data['Description'].apply(remove_single_character)



#Lemmatization
#this groups words based on their lemma ex: walk v walked v walking

def lemmatization(words):
    lemmatizer = WordNetLemmatizer()
    result = []
    for word in words:
        result.append(lemmatizer.lemmatize(word))
    return result

data['Title'] = data['Title'].apply(lemmatization)
data['Description'] = data['Description'].apply(lemmatization)

data.head()

,News Category,Title,Description
0,sports,"[russell, westbrook, tie, nba, record, unachie...","[westbrook, mindboggling, number, one, major, ..."
1,sports,"[triple, team, bad, rocket, team, quin, snyder...","[know, thought, quin, snyder’s, approach, game..."
2,sports,"[portland, trail, blazer, continue, winning, b...","[portland, trail, blazer, won, seventh, game, ..."
3,sports,"[outmanned, lightning, fall, panther]","[certainly, wasn’t, way, lightning, wanted, be..."
4,sports,"[instant, analysis, jazz, earn, 50th, win, bea...","[utah, jazz, beat, houston, rocket, one, hundr..."


# Generating Textual Features

In [5]:
import pandas as pd
import re
import collections
import nltk
nltk.download()
from textblob import TextBlob
import os

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [6]:
data['Documents'] = data['Title'] + data['Description']
data.drop(['Title','Description'],axis=1,inplace=True)
data['Documents'] = data['Documents'].apply(lambda x: ' '.join(x))
data['Documents'] = data['Documents'].apply(lambda x: x.replace("\'","").replace(',','').replace(']','').replace('[',''))
data['Word Count'] = data['Documents'].apply(lambda x: len(re.findall(r'\w+', x)))

data['Noun Phrases'] = data['Documents'].apply(lambda x: len(TextBlob(x).noun_phrases))
data['Tags'] = data['Documents'].apply(lambda t: collections.Counter(tag for word,tag in TextBlob(t).tags))

# pos tag list https://pythonprogramming.net/part-of-speech-tagging-nltk-tutorial/

data['Noun Count'] = data['Tags'].apply(lambda d: d.get('NN',0)+d.get('NNS',0)+d.get('NNP',0)+d.get('NNPS',0))
data['Adjective Count'] = data['Tags'].apply(lambda d: d.get('JJ',0)+d.get('JJR',0)+d.get('JJS',0))
data['Verb Count'] = data['Tags'].apply(lambda d: d.get('VB',0)+d.get('VBD',0)+d.get('VBG',0)+d.get('VBN',0)+d.get('VBP',0)+d.get('VBZ',0))
data.drop(['Tags'],axis=1,inplace=True)

data.head()

,News Category,Documents,Word Count,Noun Phrases,Noun Count,Adjective Count,Verb Count
0,sports,russell westbrook tie nba record unachievable ...,27,5,15,7,2
1,sports,triple team bad rocket team quin snyder keep f...,28,6,16,4,7
2,sports,portland trail blazer continue winning big one...,48,5,15,10,9
3,sports,outmanned lightning fall panther certainly was...,23,4,10,5,7
4,sports,instant analysis jazz earn 50th win beating ro...,37,6,18,6,6


# Word2vec Feature Extraction

In [7]:
import numpy as np
import pandas as pd

from gensim.models import Word2Vec
from time import time 
import multiprocessing
import logging  # logger
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

import warnings
warnings.filterwarnings('ignore')

In [9]:
cores = multiprocessing.cpu_count()

# initializing word2vec model
model = Word2Vec(min_count=20,
                     window=2, # window size for context 
                     vector_size=100,  # no of features 
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

INFO - 01:22:18: Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=100, alpha=0.03)', 'datetime': '2021-05-09T01:22:18.134286', 'gensim': '4.0.1', 'python': '3.7.5 (default, Oct 25 2019, 10:52:18) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'created'}


In [10]:
t = time()

# build vocabulary
model.build_vocab(data['Documents'], progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 01:22:18: collecting all words and their counts
WARNING - 01:22:18: Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
INFO - 01:22:18: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 01:22:18: collected 39 word types from a corpus of 9135 raw words and 48 sentences
INFO - 01:22:18: Creating a fresh vocabulary
INFO - 01:22:18: Word2Vec lifecycle event {'msg': 'effective_min_count=20 retains 25 unique words (64.1025641025641%% of original 39, drops 14)', 'datetime': '2021-05-09T01:22:18.166132', 'gensim': '4.0.1', 'python': '3.7.5 (default, Oct 25 2019, 10:52:18) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'prepare_vocab'}
INFO - 01:22:18: Word2Vec lifecycle event {'msg': 'effective_min_count=20 leaves 9042 word corpus (98.98193760262726%% of original 9135, drops 93)', 'datetime': '2021-05-09T01:22:18.166785', 'gensim': '4.0.1', '

Time to build vocab: 0.0 mins


In [11]:
t = time()

#train word2vec model 
model.train(data['Documents'], total_examples=model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 01:22:18: Word2Vec lifecycle event {'msg': 'training model with 7 workers on 25 vocabulary and 100 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2', 'datetime': '2021-05-09T01:22:18.178959', 'gensim': '4.0.1', 'python': '3.7.5 (default, Oct 25 2019, 10:52:18) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'train'}
INFO - 01:22:18: worker thread finished; awaiting finish of 6 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 5 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 4 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 3 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 2 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 1 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 0 more threads
INFO - 01:22:18: EPOCH - 1 : training on 9135 raw words (305 effective words) took 0.0s,

INFO - 01:22:18: worker thread finished; awaiting finish of 4 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 3 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 2 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 1 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 0 more threads
INFO - 01:22:18: EPOCH - 13 : training on 9135 raw words (331 effective words) took 0.0s, 51553 effective words/s
INFO - 01:22:18: worker thread finished; awaiting finish of 6 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 5 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 4 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 3 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 2 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 1 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 0

INFO - 01:22:18: worker thread finished; awaiting finish of 5 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 4 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 3 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 2 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 1 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 0 more threads
INFO - 01:22:18: EPOCH - 26 : training on 9135 raw words (323 effective words) took 0.0s, 67690 effective words/s
INFO - 01:22:18: worker thread finished; awaiting finish of 6 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 5 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 4 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 3 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 2 more threads
INFO - 01:22:18: worker thread finished; awaiting finish of 1

Time to train the model: 0.01 mins


In [12]:
def make_feature_vec(words, model, num_features):
    """
    Average the word vectors for a set of words
    """
    feature_vec = np.zeros((num_features,),dtype="float32")  # pre-initialize (for speed)
    nwords = 0.
    index2word_set = set(model.wv.index_to_key)  # words known to the model

    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            feature_vec = np.add(feature_vec,model.wv[word])
    
    feature_vec = np.divide(feature_vec, nwords)
    return feature_vec


def get_avg_feature_vecs(words, model, num_features):
    """
    Calculate average feature vectors for all headlines 
    """
    counter = 0
    feature_vecs = np.zeros((len(words),num_features), dtype='float32')  # pre-initialize (for speed)
    
    for word in words:
        feature_vecs[counter] = make_feature_vec(word, model, num_features)
        counter = counter + 1
    return feature_vecs

In [13]:
word2vec = get_avg_feature_vecs(data['Documents'], model, 100)

In [14]:
# remove instances in test set that could not be represented as feature vectors
nan_indices = list({x for x,y in np.argwhere(np.isnan(word2vec))})
if len(nan_indices) > 0:
    print('Removing {:d} instances from test set.'.format(len(nan_indices)))
    word2vec = np.delete(word2vec, nan_indices, axis=0)
    word2vec.drop(data.iloc[nan_indices, :].index, axis=0, inplace=True)
    assert word2vec.shape[0] == len(data)

In [15]:
w2v = pd.DataFrame(word2vec)

data.reset_index(drop=True, inplace=True)
w2v.reset_index(drop=True, inplace=True)
#df = pd.concat([df1, df2], axis=1)
w2v = pd.concat([data[['News Category','Word Count','Noun Phrases','Noun Count',
                                         'Adjective Count','Verb Count']],w2v],axis=1)

w2v.head()

,News Category,Word Count,Noun Phrases,Noun Count,Adjective Count,Verb Count,0,1,2,3,...,90,91,92,93,94,95,96,97,98,99
0,sports,27,5,15,7,2,-0.139191,0.172972,0.097528,0.109247,...,0.275901,0.138667,0.134518,-0.016166,0.378698,0.078404,0.222203,-0.270796,0.078494,0.137154
1,sports,28,6,16,4,7,-0.140603,0.174125,0.097980,0.108240,...,0.276365,0.139580,0.134864,-0.017174,0.379521,0.079596,0.221663,-0.271183,0.078286,0.136707
2,sports,48,5,15,10,9,-0.139865,0.173593,0.097847,0.107312,...,0.276472,0.139418,0.135319,-0.017005,0.378967,0.079099,0.222359,-0.271193,0.078933,0.136285
3,sports,23,4,10,5,7,-0.140761,0.174590,0.096981,0.106697,...,0.276483,0.140179,0.135892,-0.016939,0.379782,0.078771,0.222114,-0.270521,0.078998,0.136414
4,sports,37,6,18,6,6,-0.140718,0.174285,0.097562,0.107728,...,0.276479,0.139207,0.134532,-0.017244,0.379008,0.078861,0.221922,-0.271424,0.078358,0.136367


# Import Logistic Regression Model

In [16]:
import pickle

In [17]:
Pkl_Filename = "Pickle_RL_Model.pkl"  

with open(Pkl_Filename, 'rb') as file:  
    Pickled_LR_Model = pickle.load(file)

Pickled_LR_Model

LogisticRegression(solver='newton-cg')

# Run LR Model on Real-Time Data with W2V Features

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import OneHotEncoder

In [19]:
# X dataframe 
X_test = w2v.drop(['News Category'],axis=1) 
# y series
y_test = w2v['News Category']

In [20]:
y_test = y_test.to_numpy()
y_pred = Pickled_LR_Model.predict(X_test)

lr_acc = accuracy_score(y_test,y_pred)
lr_recall = recall_score(y_test,y_pred,average='macro')
lr_precision = precision_score(y_test,y_pred,average='macro')
lr_f1 = f1_score(y_test,y_pred,average='macro')

y_pred_roc = OneHotEncoder().fit(y_test.reshape(-1, 1)).transform(y_pred.reshape(-1,1)).toarray()
y_test_roc = OneHotEncoder().fit_transform(y_test.reshape(-1, 1)).toarray()
lr_roc = roc_auc_score(y_test_roc,y_pred_roc,multi_class='ovo')

print("Logistic Regression: ",lr_acc,lr_recall,lr_precision,lr_f1,lr_roc)

Logistic Regression:  0.22916666666666666 0.22916666666666666 0.059782608695652176 0.09482758620689655 0.48611111111111116
